# Use Stripe with agents.json
This notebook demonstrates how to use agents.json to interact with the Stripe API. This simple example shows loading the agents.json file and using it to execute Stripe tasks.


In [ ]:
%pip install agentsjson-core openai
%pip install agentsjson-integrations[stripe]

In [4]:
STRIPE_API_KEY="<your_stripe_api_key>"
OPENAI_API_KEY="<your_openai_api_key>"
agents_json_url = "https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/master/agents_json/stripe/agents.json"

### Load the agents.json file

In [ ]:
from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
import agentsjson.core as core

# load the agents.json file
data: Bundle = core.load_agents_json(agents_json_url)
flows = data.agentsJson.flows

### Define the Agent

In [5]:
query = "Create a new Stripe product with for tshirts priced at $10, $15, and $30 for small, medium, and large sizes"

In [6]:
from agentsjson.core import ToolFormat

# Format the flows data for the prompt
flows_context = core.flows_prompt(flows)

# Create the system prompt
system_prompt = f"""You are an AI assistant that helps users interact with the Stripe API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

### Configure Auth

In [9]:
from agentsjson.core.models.auth import AuthType, BearerAuthConfig
auth = BearerAuthConfig(type=AuthType.BEARER, token=STRIPE_API_KEY)

### Run the agent

In [ ]:
from openai import OpenAI
from agentsjson.core.executor import execute_flows
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    tools=core.flows_tools(flows, format=ToolFormat.OPENAI),
    temperature=0
)

response = execute_flows(response, format=core.ToolFormat.OPENAI, bundle=data, flows=flows, auth=auth)

response